- Medium

- 算是数学解法, 根据schedule逻辑出发, 答案应该是#(tasks) + #(idle slots)
- 先排最多的tasks, 然后在后面跟着的idle slots内插入其他task
- 如果idle slots不够用, 那直接在后面接上其他tasks即可, 不会打破cooldown规则, 且答案就是tasks的总数量
- 看Solution的Approach #3 Calculating Idle slots https://leetcode.com/problems/task-scheduler/solution/

In [ ]:
from collections import Counter

class Solution:
    def leastInterval(self, tasks: List[str], n: int) -> int:
        task_cnt = Counter(tasks)
        task_cnt = sorted(task_cnt.values(), reverse=True)
        max_cnt = task_cnt[0] - 1  # 最后一次做完就结束了, 不需要跟idle slots
        idle_slots = max_cnt * n  # idle slots的数量
        for cnt in task_cnt[1:]:
            idle_slots -= min(max_cnt, cnt)
            if idle_slots <= 0:  # 若idle slots不够用, 则直接返回#(tasks)
                return len(tasks)
        return len(tasks) + idle_slots

- PriorityQueue
- 以一个cooldown为周期进行循环, 按各task数量从大到小抽取
- 每次循环前有两种情况
> 1. 最大数量的task唯一, 则当前循环其第一个被抽取, 且到下一回合该task数量仍然最大 (但不一定唯一了), 仍可在第一个执行
> 2. 最大数量的task不唯一, 则这些tasks会先被抽取, 到下一回合可能仍然是最大, 可以继续如此抽取, 也可能不是最大 (说明在一个cooldown内没跑完), 那就开始抽取其余那些没跑的

In [ ]:
import heapq
from collections import Counter

class Solution:
    def leastInterval(self, tasks: List[str], n: int) -> int:
        task_cnt = Counter(tasks).values()
        pq = []
        for c in task_cnt:
            heapq.heappush(pq, -c)  # min-heap
            
        ans = 0
        while pq:
            cd = 0
            temp = []
            while cd <= n:  # 以每个cooldown周期循环
                if pq:
                    if pq[0] < -1:  # 如果不是最后一次, 那先放进temp, 循环结束后放回heap
                        temp.append(heapq.heappop(pq) + 1)
                    else:
                        heapq.heappop(pq)
                ans += 1  # 1). 做一个task, 2). 当前没task可做, idle (未进入上面if)
                if len(pq) == 0 and len(temp) == 0:  # 最后一次循环时有[1, n]个还剩1次的tasks, 所以每次heappop不会进入temp
                    break
                cd += 1
            # 放回heap
            for t in temp:
                heapq.heappush(pq, t)
        return ans